In [1]:
%pip install -q langchain-openai langchain playwright beautifulsoup4 langchain-community html2text ragas langchain-google-genai langchain_mistralai langchain-nvidia-ai-endpoints pandas

Note: you may need to restart the kernel to use updated packages.


Caso a instalação falhe pode ser devido a falta do C++ build tools, seguir as instruções do issue: https://github.com/chroma-core/chroma/issues/189#issuecomment-1454418844

In [2]:
%pip install -qU langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tqdm as notebook_tqdm
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import Html2TextTransformer
from bs4 import BeautifulSoup
from langchain.schema import Document
import os
import json
from dotenv import load_dotenv
load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

### Utilizando o html loader do langchain para capturar todos os links da pagina de providers da documentação do langchain.

In [2]:
urls = ["https://python.langchain.com/v0.2/docs/integrations/providers/"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

soup = BeautifulSoup(docs[0].page_content, 'html.parser')

# Find all <a> tags
a_tags = soup.find_all('a')

# Extract the href attributes
hrefs = [a.get('href') for a in a_tags]
filtered_hrefs = [s for s in hrefs if s.startswith("/v0.2/docs/integrations")]
unique_hrefs = list(set(filtered_hrefs))
hrefs = ['https://python.langchain.com' + s for s in unique_hrefs]



Fetching pages: 100%|##########| 1/1 [00:00<00:00,  8.53it/s]


In [4]:
hrefs

['https://python.langchain.com/v0.2/docs/integrations/document_loaders/github/',
 'https://python.langchain.com/v0.2/docs/integrations/document_loaders/grobid/',
 'https://python.langchain.com/v0.2/docs/integrations/providers/zhipuai/',
 'https://python.langchain.com/v0.2/docs/integrations/text_embedding/infinity/',
 'https://python.langchain.com/v0.2/docs/integrations/document_loaders/wikipedia/',
 'https://python.langchain.com/v0.2/docs/integrations/chat/coze/',
 'https://python.langchain.com/v0.2/docs/integrations/vectorstores/faiss/',
 'https://python.langchain.com/v0.2/docs/integrations/tools/google_search/',
 'https://python.langchain.com/v0.2/docs/integrations/providers/college_confidential/',
 'https://python.langchain.com/v0.2/docs/integrations/providers/oracleai/',
 'https://python.langchain.com/v0.2/docs/integrations/retrievers/',
 'https://python.langchain.com/v0.2/docs/integrations/stores/upstash_redis/',
 'https://python.langchain.com/v0.2/docs/integrations/providers/bage

### Capturando somente os trechos que interessam

In [3]:
def extract_content(string: str):
    start_phrase = "On this page"
    end_phrase = "Edit this page"
    
    # Find the starting index of the start_phrase
    start_index = string.find(start_phrase)
    if start_index == -1:
        return ""  # If start_phrase not found, return empty string
    
    # Find the ending index of the end_phrase
    end_index = string.find(end_phrase, start_index)
    if end_index == -1:
        return ""  # If end_phrase not found, return empty string
    
    # Extract content between the start and end phrases
    return string[start_index + len(start_phrase):end_index].strip()

### Carregar o conteúdo de todos os links

In [4]:
loader = AsyncHtmlLoader(hrefs)
docs = loader.load()

Fetching pages: 100%|##########| 956/956 [02:33<00:00,  6.23it/s]


In [5]:
html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

Capturar o page content dos documentos

In [6]:
for i in range(len(docs_transformed)):
    result = extract_content(str(docs_transformed[i]))
    docs_transformed[i].page_content = result

In [7]:
#example
docs_transformed[0].metadata.get('description')

"Cloud SQL is a fully managed relational database service that offers high performance, seamless integration, and impressive scalability. It offers MySQL, PostgreSQL, and SQL Server database engines. Extend your database application to build AI-powered experiences leveraging Cloud SQL's Langchain integrations."

### Salvando todo os documentos em um arquivo para reutilizar posteriormente

In [8]:
documents_dict = [
    {"page_content": doc.page_content, "metadata": doc.metadata} for doc in docs_transformed
]

# Save the list of dictionaries to a JSON file
with open("input/documents_langchain.json", "w") as file:
    json.dump(documents_dict, file, indent=4)

print("Documents have been saved to 'documents_langchain.json'")

Documents have been saved to 'documents_langchain.json'


### Carregando os arquivos salvos 

In [11]:
# Load the list of dictionaries from the JSON file
with open("input/documents_langchain.json", "r") as file:
    documents_dict = json.load(file)

# Convert the list of dictionaries back to a list of Document objects
documents = [
    Document(page_content=doc["page_content"], metadata=doc["metadata"])
    for doc in documents_dict
]


### Documents split

In [10]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=10)

split_documents = []
for doc in documents:
    split_docs = text_splitter.split_documents([doc])
    split_documents.extend(split_docs)

### Save documents split

In [11]:
documents_dict = [
    {"page_content": doc.page_content, "metadata": doc.metadata} for doc in split_documents
]

# Save the list of dictionaries to a JSON file
with open("input/documents_split_langchain.json", "w") as file:
    json.dump(documents_dict, file, indent=4)

print("Documents have been saved to 'documents_split_langchain.json'")

Documents have been saved to 'documents_split_langchain.json'


### Load documents split

In [2]:
# Load the list of dictionaries from the JSON file
with open("input/documents_split_langchain.json", "r") as file:
    documents_dict = json.load(file)

# Convert the list of dictionaries back to a list of Document objects
documents = [
    Document(page_content=doc["page_content"], metadata=doc["metadata"])
    for doc in documents_dict
]
